In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from MySQLConnection import * 
import mysql_auth

# Inter Skill Interval Analysis

In [4]:
# primitive data 
sqlconn = MySQLConnection()
all_table_names = sqlconn.get_table_names()
table_names = [x for x in all_table_names if ((x.startswith('202105')) or (x.startswith('202106')))]
sql_union = ''
for table in table_names:
    sql = f"""
        SELECT `MatchId`, `Map`, `Section`, `Timestamp`, `Team`, `Player`, `Hero`, 
       `Cooldown1%%/s`, `Cooldown2%%/s`, `CooldownCrouching%%/s`, `CooldownSecondaryFire%%/s`, `UltimatesUsed/s`, `TF_order`, `TF_winner`, `RCP` from `{table}`
        """
    if table is table_names[-1]:
        sql_union = sql_union + sql + ';'
    else:
        sql_union = sql_union + sql + ' UNION '

input_df = sqlconn.read_table_with_sql(sql_union)

In [22]:
# set hero1 and hero2
hero1 = 'Ana'
hero2 = 'Winston'

# set skill1 and skill2 
skill_list = ['Cooldown1%/s', 'Cooldown2%/s', 'CooldownCrouching%/s', 'CooldownSecondaryFire%/s', 'UltimatesUsed/s']
skill1 = 'Cooldown2%/s'
skill2 = 'Cooldown1%/s'

# filtering
cond1 = ((input_df['Hero']==hero1) & (input_df[skill1] > 0))
cond2 = ((input_df['Hero']==hero2) & (input_df[skill2] > 0))
df_init = input_df.loc[(cond1 | cond2), :]

# grouping 
df_group = df_init.set_index(['MatchId', 'Map', 'Section', 'Team'])[['Timestamp', 'Hero', skill1, skill2, 'TF_order', 'TF_winner', 'RCP']]
df_group.loc[df_group[skill1]>0, skill1] = 1
df_group.loc[df_group[skill1]<0, skill1] = 0
df_group.loc[df_group[skill2]>0, skill2] = 1
df_group.loc[df_group[skill2]<0, skill2] = 0
display(df_group)

# get ISI
# to_drop = (df_group['B'] == df_group['B'].shift())&(df_group['C']==df_group['C'].shift())
# df = df[~to_drop]

# plot

Timestamp     Hero  \
MatchId     Map                 Section Team                       
20210513_01 Eichenwalde         0       GZC        2.02  Winston   
                                        GZC       14.11  Winston   
                                        NYE       26.21      Ana   
                                        NYE       26.21  Winston   
                                        GZC       35.28  Winston   
...                                                 ...      ...   
20210605_01 Volskaya Industries 0       SEO      108.87      Ana   
                                        SEO      128.02      Ana   
                                        SEO      140.12      Ana   
                                        SEO      151.21      Ana   
                                        SEO      189.51      Ana   

                                              Cooldown2%/s  Cooldown1%/s  \
MatchId     Map                 Section Team                               
20210513_01 Eichenwalde         0       GZC            0.0           1.0   
                                        GZC            0.0           1.0   
                                        NYE            1.0           0.0   
                                        NYE            0.0           1.0   
                                        GZC            0.0           1.0   
...                                                    ...           ...   
20210605_01 Volskaya Industries 0       SEO            1.0           0.0   
                                        SEO            1.0           0.0   
                                        SEO            1.0           0.0   
                                        SEO            1.0           0.0   
                                        SEO            1.0           0.0   

                                              TF_order TF_winner       RCP  
MatchId     Map                 Section Team                                
20210513_01 Eichenwalde         0       GZC        NaN      None  0.000000  
                                        GZC        NaN      None  0.000000  
                                        NYE        1.0       NYE  0.000000  
                                        NYE        1.0       NYE  0.000000  
                                        GZC        1.0       NYE  0.000000  
...                                                ...       ...       ...  
20210605_01 Volskaya Industries 0       SEO        3.0       SEO  0.000000  
                                        SEO        4.0       SEO  0.000000  
                                        SEO        4.0       SEO  0.000000  
                                        SEO        4.0       SEO -5.333333  
                                        SEO        5.0       NYE  1.833333  

[21238 rows x 7 columns]